In [1]:
# nb-svf-all-segs.ipynb

# <font color="navy">Рассмотрим модель SVD (Singular Value Decomposition).

<font color="darkgreen">Работает с python 3.11 и 3.9

In [2]:
import sys

In [3]:
import numpy as np
import pandas as pd
import time
import warnings
import random
from collections import defaultdict
from surprise import SVD, Reader, Dataset, accuracy #, KNNBasic, NMF, , , accuracy
from surprise.model_selection import train_test_split as sp_train_test_split
from tqdm.notebook import tqdm
#warnings.simplefilter("ignore")
from IPython.display import Markdown, display

In [4]:
print(f'System version: {sys.version}')
print(f'Pandas version: {pd.__version__}')
print(f'Numpy version:  {np.__version__}')

System version: 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]
Pandas version: 1.5.3
Numpy version:  1.23.5


In [5]:
from settings import *
DEBUG = True

.\cache\2019-Nov.csv
.\cache\2019-Oct.csv


In [6]:
start_time = time.time()

In [7]:
pd.options.display.float_format = "{:,.2f}".format
warnings.simplefilter("ignore")
%matplotlib inline

## Создание рекомендательных систем с использованием библиотеки Surprise

In [8]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [9]:
def toratings(ds):
    """
    Преобразует таблицу к строкам: user_id, category_name, rating
    """
    cols = np.sort(dataset[SPLIT_CATEGORY].unique()).tolist() # Получаем значения всех категорий
    ds = ds[cols]
    #ds = ds.loc[:, (ds != 0).any(axis=0)] # Удаляем столбцы с одними нулями
    ds.reset_index(inplace=True) # Преобразуем индекс в столбец

    ratings = pd.DataFrame()
    #cols = list(ds.columns)
    if 'user_id' in cols:
        cols.remove('user_id')
    for col in cols:
        if col not in list(ds.columns):
            cols.remove(col)
    for col in cols:
        dtmp = ds[ds[col]!=0][['user_id',col]].rename(columns={col: 'rating'})
        dtmp.insert(1,'category',col)
        ratings=pd.concat([ratings,dtmp], ignore_index = True)
    return ratings

In [10]:
dataset = pd.read_pickle(ds_flt_file) # Полные данные после фильтрации
ds = pd.read_pickle(showcase_seg_file) # Сегментированная витрина

In [11]:
%%time
# Главный цикл
random.seed(42)
prec = []
for seg in set(ds['Segment']):
    print(f'Сегмент: {seg}')
    df = ds[ds['Segment']==seg]
    df = toratings(df)
        
    reader = Reader(rating_scale=(1, df['rating'].max()))
    data = Dataset.load_from_df(df, reader)

    # Разработка рекомендательной системы с использованием Surprise
    # Разделение данных на обучающий и тестовый наборы
    trainset, testset = sp_train_test_split(data, test_size=0.2, random_state=42)
    # Создание модели SVD
    model = SVD(n_factors=100, n_epochs=500, lr_all=0.005, reg_all=0.02, random_state=42)
    # Обучение модели на обучающем наборе
    model.fit(trainset)
    # Then predict ratings for all pairs (u, i) that are NOT in the training set.
    testset = trainset.build_anti_testset()
    predictions = model.test(testset)
    top_n = get_top_n(predictions, n=5)
    
    # Получение прогнозов модели на тестовом наборе

    # Расчет MAE и RMSE
    mae = accuracy.mae(predictions, False)
    rmse = accuracy.rmse(predictions, False)
    prec.append([seg, mae, rmse])
    
    #print(f'MAE: {mae}')
    #print(f'RMSE: {rmse}')

    # Выдача рекомендаций на основе обученной модели
    # После успешного обучения модели мы можем использовать ее для выдачи рекомендаций. 
    # Например, для пользователя с идентификатором user_id, мы можем получить топ N рекомендаций.
    # Получение топ 5 рекомендаций для пользователя с идентификатором user_id
    for id in random.sample(range(0, len(predictions)), 2):
        user_id = predictions[id][0]
        top_n = get_top_n(predictions, n=5)
        # Вывод рекомендаций
        for user, user_ratings in top_n.items():
            if user == user_id:
                print(f'\nПользователь: {user_id}')
                print(f'Рекомендовано: {[item_id for item_id, rating in user_ratings]}')
                print(f'Куплено: {set(dataset[(dataset["user_id"]==user_id) & (dataset["event_type"]=="purchase")]["category_code_level2"])}')
    display(Markdown('---'))
prec = pd.DataFrame(prec,columns=['Segment', 'RME', 'RMSE'])
prec.set_index('Segment', inplace=True)

Сегмент: 0

Пользователь: 553932996
Рекомендовано: ['tv', 'water_heater', 'cooler', 'microphone', 'wallet']
Куплено: {'microwave', 'notebook'}

Пользователь: 521565698
Рекомендовано: ['notebook', 'cooler', 'dolls', 'shirt', 'juicer']
Куплено: {'subwoofer', 'tv'}


---

Сегмент: 1

Пользователь: 512438522
Рекомендовано: ['tv', 'winch', 'parktronic', 'fan', 'ebooks']
Куплено: {'desktop', 'headphone'}

Пользователь: 541350829
Рекомендовано: ['winch', 'fan', 'tv', 'parktronic', 'massager']
Куплено: {'shoes'}


---

Сегмент: 2

Пользователь: 557438880
Рекомендовано: ['desktop', 'trainer', 'cartrige', 'mouse', 'keyboard']
Куплено: {'power_supply', 'motherboard', 'monitor', 'cpu', 'videocards', 'memory', 'hdd'}

Пользователь: 553804520
Рекомендовано: ['monitor', 'desktop', 'cartrige', 'trainer', 'hdd']
Куплено: {'videocards', 'headphone'}


...

Сегмент: 11

Пользователь: 559394618
Рекомендовано: ['compressor', 'tv', 'steam_cooker', 'camera', 'video']
Куплено: {'acoustic', 'headphone'}

Пользователь: 512980976
Рекомендовано: ['steam_cooker', 'tv', 'video', 'moccasins', 'headphone']
Куплено: {'clocks', 'notebook', 'desktop', 'photo'}


---

Сегмент: 12

Пользователь: 568069892
Рекомендовано: ['clocks', 'headphone', 'microwave', 'carriage', 'toys']
Куплено: {'saw'}

Пользователь: 553013818
Рекомендовано: ['headphone', 'clocks', 'carriage', 'microwave', 'printer']
Куплено: {'bicycle'}


---

CPU times: total: 15.2 s
Wall time: 14.8 s


In [12]:
from user_func import prdf
prdf(prec.T)
print('Finish')

Segment,0,1,2,3,4,5,6,7,8,9,10,11,12
RME,0.53,0.48,0.45,0.88,0.33,0.46,0.37,0.49,0.47,0.38,0.42,0.55,0.65
RMSE,0.66,0.62,0.58,1.11,0.44,0.58,0.48,0.62,0.58,0.49,0.53,0.67,0.79


Finish


In [13]:
seconds=int(time.time() - start_time)
print("%d:%02d" % (seconds//60, seconds-60*(seconds//60),))

0:16
